### Definitions:
* Tokens: words, syllables, punctuation... Basic units of speech.
* Characters: letters, numbers, spaces, punctuations, tabs...
    * Models will be built on tokens/words or characters.
* Vocabulary: the set of all words that I consider in the model. The X most common words (10K/100K or whatever number)
* Corpus: The set of all our texts. Here it's the dataset.
* N-gram: a set of N consecutive items (tokens/words, subwords, characters...). They are particularized as unigrams, bigrams, trigrams...
* Vectors: We can represent a word by the position in the vocabulary ("Cipote" = 41564) or by a vector in a vector space of dimension n ("Cipote" = [0.1, 0.324, .004, ..., 0.34])
aaa
